In [5]:
# Helpers
import numpy as np

# DB 
import psycopg2
from django.conf import settings

# Evaluation
from sklearn.metrics import roc_curve, auc, f1_score

# Plot
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
connection = psycopg2.connect(user = settings.DATABASES["default"]["USER"],
                                  password = settings.DATABASES["default"]["PASSWORD"],
                                  host = settings.DATABASES["default"]["HOST"],
                                  port = settings.DATABASES["default"]["PORT"],
                                  database = settings.DATABASES["default"]["NAME"])
connection.autocommit=True
cursor = connection.cursor()

In [3]:
%%time
query = """select experiment_id, sp_test, sp_hat, auc_threshold
from EDM2020_2020_02_19 where method='nmf' and attempts_train = 1"""
cursor.execute(query)
row = cursor.fetchone()

CPU times: user 1.5 s, sys: 2.64 s, total: 4.14 s
Wall time: 1min 30s


In [7]:
# %%time
update_queries = []
total = 0
while row:
    exp_id, sp_test, sp_hat, auc_threshold = row
    sp_test = np.asarray(sp_test)
    sp_hat = np.asarray(sp_hat)
    
    # Get train predicted values
    # 0 is when there aren't any attempts
    y = sp_test[np.where(sp_test > 0)]
    y_pred = sp_hat[np.where(sp_test > 0)]
    rmse = np.sqrt(np.mean(np.power(y - y_pred, 2))) 
    
    update = {
        "test_rmse": rmse,
    }
    
    if auc_threshold:
        # Binarize prediction
        y[np.where(y == 0.1)] = 0
        y_pred_binary = np.where(np.asarray(y_pred) > auc_threshold, 1, 0)

        # Calculate accuracy and RMSE
        acc = np.logical_not(np.logical_xor(y, y_pred_binary)).sum()/len(y)
        f1 = f1_score(y, y_pred_binary)
        
        update.update({
            "test_acc": acc,
            "test_1":  np.where(y == 1)[0].shape[0],
            "test_0":  np.where(y == 0)[0].shape[0],
            "test_pred_1": np.where(y_pred_binary == 1)[0].shape[0],
            "test_pred_0": np.where(y_pred_binary == 0)[0].shape[0],
            "test_f1": f1
        })
            
   # Write PSQL query
    update_query = "UPDATE EDM2020_2020_02_19 SET "
    update_list = []
    query_values = []
    for key, value in update.items():
        update_list.append(key + "= %s")
        query_values.append(value)
    
    update_query += ", ".join(update_list) + "where experiment_id = %s"
    query_values.append(exp_id)
    query_values = tuple(query_values)
    query = cursor.mogrify(update_query, query_values)
    update_queries.append(query)
    total += 1
    try:
        row = cursor.fetchone()
    except psycopg2.ProgrammingError:
        row = False

In [8]:
for query in update_queries:
    cursor.execute(query)

In [9]:
total

8550